# 1. Setup

In [1]:
%%capture
!pip install -q yt-dlp pydub ffmpeg-python openai-whisper tqdm

In [2]:
import os
import csv
import logging

logging.basicConfig(
    filename="pipeline_errors.log",
    level=logging.ERROR,
    format="%(asctime)s\t%(levelname)s\t%(message)s"
)

csv_path = "/kaggle/input/small-andrew-tate/fetched.csv"

entries = []
with open(csv_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        vid = row.get("videoId")
        url = row.get("url")
        if vid and url:
            entries.append({"id": vid, "url": url})


some_entries = entries[:]

some_entries

[{'id': 'gEe4XhIiUDI', 'url': 'https://www.youtube.com/watch?v=gEe4XhIiUDI'},
 {'id': 'TgeE6Cr02nc', 'url': 'https://www.youtube.com/watch?v=TgeE6Cr02nc'},
 {'id': 'sjw-lw-ZT4g', 'url': 'https://www.youtube.com/watch?v=sjw-lw-ZT4g'},
 {'id': 'YsL9FdQwOsU', 'url': 'https://www.youtube.com/watch?v=YsL9FdQwOsU'},
 {'id': '3ifgUkVOKaE', 'url': 'https://www.youtube.com/watch?v=3ifgUkVOKaE'},
 {'id': 'c0ztBJKmxKI', 'url': 'https://www.youtube.com/watch?v=c0ztBJKmxKI'},
 {'id': 'kCyAZ6wYg8w', 'url': 'https://www.youtube.com/watch?v=kCyAZ6wYg8w'},
 {'id': 'oT2WEG72iFY', 'url': 'https://www.youtube.com/watch?v=oT2WEG72iFY'},
 {'id': '5Y5d3EvwRvM', 'url': 'https://www.youtube.com/watch?v=5Y5d3EvwRvM'},
 {'id': '_M_9tkThiPU', 'url': 'https://www.youtube.com/watch?v=_M_9tkThiPU'}]

In [3]:
from yt_dlp import YoutubeDL
from pydub import AudioSegment
from pydub.utils import make_chunks
import whisper
from tqdm import tqdm

def download_m4a(url, output_dir="audio"):
    os.makedirs(output_dir, exist_ok=True)
    opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(output_dir, "audio.%(ext)s"),
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "m4a"
        }],
        "quiet": False,
        "no_warnings": True
    }
    with YoutubeDL(opts) as ydl:
        ydl.download([url])
    return os.path.join(output_dir, "audio.m4a")

def convert_to_wav(m4a_path):
    wav_path = os.path.splitext(m4a_path)[0] + ".wav"
    AudioSegment.from_file(m4a_path).export(wav_path, format="wav")
    return wav_path

def split_audio(wav_path, chunk_ms=60000):
    audio = AudioSegment.from_wav(wav_path)
    chunks = make_chunks(audio, chunk_ms)
    base = os.path.dirname(wav_path)
    paths = []
    for i, c in enumerate(chunks):
        p = os.path.join(base, f"chunk_{i}.wav")
        c.export(p, format="wav")
        paths.append(p)
    return paths

def cleanup_files(*paths):
    for p in paths:
        try:
            if os.path.exists(p):
                os.remove(p)
        except Exception as e:
            logging.error(f"CLEANUP_ERROR\t{p}\t{e}")


# 3. Pipeline

In [4]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 30.1MiB/s]


In [5]:
for entry in tqdm(some_entries, desc="Videos"):
    vid, url = entry["id"], entry["url"]
    m4a_path = None
    wav_path = None
    chunks = []

    try:
        m4a_path = download_m4a(url)
        wav_path = convert_to_wav(m4a_path)
        chunks = split_audio(wav_path, chunk_ms=60000)
        
        # Transcribe
        texts = []
        for chunk in tqdm(chunks, desc=f"Transcribing {vid}", leave=False):
            res = model.transcribe(chunk)
            texts.append(res["text"])
        
        # Save transcript
        full_text = " ".join(texts)
        out_txt = f"{vid}.txt"
        with open(out_txt, "w", encoding="utf-8") as f:
            f.write(full_text)
        
        print(f"✓ Completed {vid}")
        
    except Exception as e:
        logging.error(f"{vid}\t{url}\tERROR\t{e}")
    
    finally:
        # Clean up all files regardless of success or failure
        if m4a_path and os.path.exists(m4a_path):
            os.remove(m4a_path)
        if wav_path and os.path.exists(wav_path):
            os.remove(wav_path)
        for chunk in chunks:
            if os.path.exists(chunk):
                os.remove(chunk)

print("✅ Done! Check pipeline_errors.log for any errors.")

Videos:   0%|          | 0/10 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=gEe4XhIiUDI
[youtube] gEe4XhIiUDI: Downloading webpage
[youtube] gEe4XhIiUDI: Downloading tv client config
[youtube] gEe4XhIiUDI: Downloading player 461f4c95-main
[youtube] gEe4XhIiUDI: Downloading tv player API JSON
[youtube] gEe4XhIiUDI: Downloading ios player API JSON
[youtube] gEe4XhIiUDI: Downloading m3u8 information
[info] gEe4XhIiUDI: Downloading 1 format(s): 251
[download] Destination: audio/audio.webm
[download] 100% of   70.11MiB in 00:00:05 at 12.88MiB/s  
[ExtractAudio] Destination: audio/audio.m4a
Deleting original file audio/audio.webm (pass -k to keep)



Transcribing gEe4XhIiUDI:   0%|          | 0/72 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing gEe4XhIiUDI:   1%|▏         | 1/72 [00:12<14:30, 12.25s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing gEe4XhIiUDI:   3%|▎         | 2/72 [00:25<14:41, 12.60s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing gEe4XhIiUDI:   4%|▍         | 3/72 [00:38<15:07, 13.16s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 inst

✓ Completed gEe4XhIiUDI


Videos:  10%|█         | 1/10 [18:55<2:50:23, 1135.96s/it]

...

✓ Completed 5Y5d3EvwRvM


Videos:  90%|█████████ | 9/10 [1:50:44<12:27, 747.52s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=_M_9tkThiPU
[youtube] _M_9tkThiPU: Downloading webpage
[youtube] _M_9tkThiPU: Downloading tv client config
[youtube] _M_9tkThiPU: Downloading tv player API JSON
[youtube] _M_9tkThiPU: Downloading ios player API JSON
[youtube] _M_9tkThiPU: Downloading m3u8 information
[info] _M_9tkThiPU: Downloading 1 format(s): 251
[download] Destination: audio/audio.webm
[download] 100% of   19.13MiB in 00:00:00 at 28.02MiB/s  
[ExtractAudio] Destination: audio/audio.m4a
Deleting original file audio/audio.webm (pass -k to keep)



Transcribing _M_9tkThiPU:   0%|          | 0/21 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing _M_9tkThiPU:   5%|▍         | 1/21 [00:12<04:07, 12.35s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing _M_9tkThiPU:  10%|▉         | 2/21 [00:25<04:01, 12.69s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Transcribing _M_9tkThiPU:  14%|█▍        | 3/21 [00:39<04:03, 13.55s/it]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 inst

✓ Completed _M_9tkThiPU
✅ Done! Check pipeline_errors.log for any errors.


In [6]:
print("Hello World!")

Hello World!
